In [1]:
##Making Some random changes to this code to see if these changes are transferred to Github
import pandas as pd 
import numpy as np
from datetime import datetime
import re

In [2]:
# reading csv file  
raw = pd.read_csv("SAM_export/input/ContractOpportunitiesFullCSV.csv", encoding='mac_roman', error_bad_lines=False) #import all the solicitations
SearchAgents = pd.read_csv("SAM_export/input/Codes_new.csv") #import list of search agents
NAICSCode = pd.read_csv("SAM_export/input/NAICSCode_new.csv")
#RED WINDOW WILL APPEAR IF THERE ARE ANY BAD LINE. CAN IGNORE

C:\Users\Sravan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (17,27) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


###  DO NOT RUN THE LINE BELOW FOR THE FIRST TIME IN A NEW FY

In [3]:
existingSol = pd.read_csv("SAM_export/input/existingSol.csv") # DO NOT Run this for the first time in a new FY
#this dateframe retains the solicitations previously process during this FY, so do not need for the first run in a new FY

In [4]:
raw['date'] = raw['PostedDate'].str.extract('(....-..-..)', expand=True) #convert the string into datetime format
raw['date'] = pd.to_datetime(raw['date'], errors='coerce')
raw['year'], raw['month'] = raw['date'].dt.year, raw['date'].dt.month #create a seperate column for year and month
#raw.set_index('date', inplace=True)
raw

,NoticeId,Title,Sol#,Department/Ind.Agency,CGAC,Sub-Tier,FPDS Code,Office,AAC Code,PostedDate,...,State,City,ZipCode,CountryCode,AdditionalInfoLink,Link,Description,date,year,month
0,6942b4d8d0d446c09419119428e447c2,Whirltown 1 Low & Medium Switchgear,W912NW20Q0023,DEPT OF DEFENSE,21.0,DEPT OF THE ARMY,2100,W6QK CCAD CONTR OFF,W912NW,2021-06-30 22:45:50.059-04,...,TX,CORPUS CHRISTI,78419-5260,USA,NaN,https://beta.sam.gov/opp/6942b4d8d0d446c094191...,Annual Preventative Maintenance and Technical ...,2021-06-30,2021,6
1,935d153a1f54463f84757df2e621b48c,"National Guard Bureau, Region 5, Architect and...",W9133L-21-R-6005,DEPT OF DEFENSE,21.0,DEPT OF THE ARMY,2100,W39L USA NG READINESS CENTER,W9133L,2021-06-30 22:44:52.743-04,...,VA,ARLINGTON,22204-1382,USA,NaN,https://beta.sam.gov/opp/935d153a1f54463f84757...,30 June 2021:† †See attached Amendment W9133L-...,2021-06-30,2021,6
2,a0ba5dd93d6e48a7a453c6f5c7e0d415,Integrated Solid Waste Management Services for...,N40084-21-R-2003,DEPT OF DEFENSE,17.0,DEPT OF THE NAVY,1700,NAVAL FAC ENGINEERING CMD FAR EAST,N40084,2021-06-30 22:40:25.859-04,...,AP,FPO,96349-0013,USA,NaN,https://beta.sam.gov/opp/a0ba5dd93d6e48a7a453c...,"The Contractor shall provide all labor, superv...",2021-06-30,2021,6
3,be61a70518a74dc7a13c2b00f0d31153,Awarded a Purchase Order for the evaluation an...,NW-WG3401-21-111SN,"COMMERCE, DEPARTMENT OF",13.0,NATIONAL OCEANIC AND ATMOSPHERIC ADMINISTRATION,1330,NATIONAL OCEANIC AND ATMOSPHERIC ADMINISTRATION,NaN,2021-06-30 22:39:37.138-04,...,NaN,NaN,NaN,US,NaN,https://beta.sam.gov/opp/be61a70518a74dc7a13c2...,Special Notice:† The U.S. Department of Commer...,2021-06-30,2021,6
4,363ce7cf9bfb4f1082057cabf8db4339,Roofing and Exterior Painting IDIQ Contract at...,N4008421B5500,DEPT OF DEFENSE,17.0,DEPT OF THE NAVY,1700,NAVAL FAC ENGINEERING CMD FAR EAST,N40084,2021-06-30 22:37:34.924-04,...,AP,FPO,96349-0013,USA,NaN,https://beta.sam.gov/opp/363ce7cf9bfb4f1082057...,Amendment 0004†is†issued on†01 Jul 2021. =====...,2021-06-30,2021,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179054,e5d82111eaced6a57db622e5e9a998ca,61--SOLE SOURCE PURCHASE FOR SCHWEITZER RELAYS,DE-RQ65-02WJ59102,NaN,NaN,NaN,NaN,NaN,NaN,2001-12-17 00:00:00-05,...,NaN,NaN,NaN,NaN,NaN,https://beta.sam.gov/opp/e5d82111eaced6a57db62...,The Western Area Power Administration intends ...,2001-12-17,2001,12
179055,40e98bbcd009386db3486745fa01180c,R--Transportation Safeguards Training Center (...,DE-RP04-02AL67577,NaN,NaN,NaN,NaN,NaN,NaN,2001-11-09 00:00:00-05,...,NaN,NaN,NaN,NaN,NaN,https://beta.sam.gov/opp/40e98bbcd009386db3486...,DOE/AL will be procuring a range of logistical...,2001-11-09,2001,11
179056,55dd315c0ac10ce8bcfdfb2b33d4b900,R--DOE-Wide NEPA Contracts,DE-RP04-02AL67464,NaN,NaN,NaN,NaN,NaN,NaN,2001-10-31 00:00:00-05,...,NaN,NaN,NaN,NaN,NaN,https://beta.sam.gov/opp/55dd315c0ac10ce8bcfdf...,Small Business Sources Sought for the Preparat...,2001-10-31,2001,10
179057,ed172bb4789985845b0c44f55c2bf102,R--DOE-Wide NEPA Contracts,DE-RP04-02AL67464,NaN,NaN,NaN,NaN,NaN,NaN,2001-10-31 00:00:00-05,...,NaN,NaN,NaN,NaN,NaN,https://beta.sam.gov/opp/ed172bb4789985845b0c4...,Small Business Sources Sought for the Preparat...,2001-10-31,2001,10


### FILTER THE DATES BELOW FOR CURRENT FY

In [5]:
raw['date'] = pd.to_datetime(raw['date'])

# Change the start and end date when it's a new FY

In [6]:
start_date = '2020-10-01'
end_date = '2021-09-30'

In [7]:
mask = (raw['date'] >= start_date) & (raw['date'] <= end_date)
relevantSol = raw.loc[mask]
relevantSol.set_index('date', inplace=True)
relevantSol

,NoticeId,Title,Sol#,Department/Ind.Agency,CGAC,Sub-Tier,FPDS Code,Office,AAC Code,PostedDate,...,OrganizationType,State,City,ZipCode,CountryCode,AdditionalInfoLink,Link,Description,year,month
date,,,,,,,,,,,,,,,,,,,,,
2021-06-30,6942b4d8d0d446c09419119428e447c2,Whirltown 1 Low & Medium Switchgear,W912NW20Q0023,DEPT OF DEFENSE,21.0,DEPT OF THE ARMY,2100,W6QK CCAD CONTR OFF,W912NW,2021-06-30 22:45:50.059-04,...,OFFICE,TX,CORPUS CHRISTI,78419-5260,USA,NaN,https://beta.sam.gov/opp/6942b4d8d0d446c094191...,Annual Preventative Maintenance and Technical ...,2021,6
2021-06-30,935d153a1f54463f84757df2e621b48c,"National Guard Bureau, Region 5, Architect and...",W9133L-21-R-6005,DEPT OF DEFENSE,21.0,DEPT OF THE ARMY,2100,W39L USA NG READINESS CENTER,W9133L,2021-06-30 22:44:52.743-04,...,OFFICE,VA,ARLINGTON,22204-1382,USA,NaN,https://beta.sam.gov/opp/935d153a1f54463f84757...,30 June 2021:† †See attached Amendment W9133L-...,2021,6
2021-06-30,a0ba5dd93d6e48a7a453c6f5c7e0d415,Integrated Solid Waste Management Services for...,N40084-21-R-2003,DEPT OF DEFENSE,17.0,DEPT OF THE NAVY,1700,NAVAL FAC ENGINEERING CMD FAR EAST,N40084,2021-06-30 22:40:25.859-04,...,OFFICE,AP,FPO,96349-0013,USA,NaN,https://beta.sam.gov/opp/a0ba5dd93d6e48a7a453c...,"The Contractor shall provide all labor, superv...",2021,6
2021-06-30,be61a70518a74dc7a13c2b00f0d31153,Awarded a Purchase Order for the evaluation an...,NW-WG3401-21-111SN,"COMMERCE, DEPARTMENT OF",13.0,NATIONAL OCEANIC AND ATMOSPHERIC ADMINISTRATION,1330,NATIONAL OCEANIC AND ATMOSPHERIC ADMINISTRATION,NaN,2021-06-30 22:39:37.138-04,...,AGENCY,NaN,NaN,NaN,US,NaN,https://beta.sam.gov/opp/be61a70518a74dc7a13c2...,Special Notice:† The U.S. Department of Commer...,2021,6
2021-06-30,363ce7cf9bfb4f1082057cabf8db4339,Roofing and Exterior Painting IDIQ Contract at...,N4008421B5500,DEPT OF DEFENSE,17.0,DEPT OF THE NAVY,1700,NAVAL FAC ENGINEERING CMD FAR EAST,N40084,2021-06-30 22:37:34.924-04,...,OFFICE,AP,FPO,96349-0013,USA,NaN,https://beta.sam.gov/opp/363ce7cf9bfb4f1082057...,Amendment 0004†is†issued on†01 Jul 2021. =====...,2021,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-01,7f414fb3b4d94c0f9157a9a6f72f37d7,1560012663547SY,SPE4A721R0025SY,DEPT OF DEFENSE,97.0,DEFENSE LOGISTICS AGENCY (DLA),97AS,DLA AVIATION,SPE4A7,2020-10-01 07:13:29.946-04,...,OFFICE,VA,RICHMOND,23297,USA,NaN,https://beta.sam.gov/opp/7f414fb3b4d94c0f9157a...,This is a single National Stock Number (NSN) R...,2020,10
2020-10-01,f1c901ab5d7a4c6f8c0db1b13524ff77,1680015636028SY,SPE4A721R0022,DEPT OF DEFENSE,97.0,DEFENSE LOGISTICS AGENCY (DLA),97AS,DLA AVIATION,SPE4A7,2020-10-01 07:06:19.479-04,...,OFFICE,VA,RICHMOND,23297,USA,NaN,https://beta.sam.gov/opp/f1c901ab5d7a4c6f8c0db...,This is a single National Stock Number (NSN) R...,2020,10
2020-10-01,2c89437f747442fb9ac3e87eb26f1479,66--GAGE KIT,N0010421QBA35,DEPT OF DEFENSE,17.0,DEPT OF THE NAVY,1700,NAVSUP WEAPON SYSTEMS SUPPORT MECH,N00104,2020-10-01 07:00:56.561-04,...,OFFICE,PA,MECHANICSBURG,17050-0788,USA,NaN,https://beta.sam.gov/opp/2c89437f747442fb9ac3e...,"NSN 1H-6685-015107189-X3, TDP VER 002, QTY 5 E...",2020,10


In [8]:
#Number of Total Solicitations in the FY
relevantSol['Sol#'].nunique()

29891

In [9]:
#Need to filter out the relevant solitication by NAICS codes, Classification codes, and description columns
#First do it by NAICS codes
NAICS = SearchAgents['NaicsCode']
NAICS = NAICS.dropna()
Class = SearchAgents['ClassificationCode']
Class = Class.dropna()
keyWords = SearchAgents['KeyWords']
keyWords = keyWords.dropna()
existingSol = existingSol.drop(columns=['Unnamed: 0']) #DO NOT run this for the first time in a FY
#keyWords

In [10]:
existingSol

,Sol#
0,19AQMM20Q0173
1,4564DRFL00023
2,FORTBRAGG4THBN3RDSFGDC0001
3,165THINBRIGADE00013
4,N66001-20-R-0024
...,...
1444,W912QR-INDIANAPOLIS(KENT)INFY21
1445,W912QR-BentonHarborMIFY21
1446,W912QR-WESTLANDMIFY21
1447,W912QR-MASONOHFY21


In [11]:
#looking for related NAICS, Classification codes and keywords
relevantSol_Codes = relevantSol[relevantSol['NaicsCode'].isin(NAICS) & relevantSol['ClassificationCode'].isin(Class)]
relevantSol_keyWords=relevantSol[relevantSol.Description.str.contains('|'.join(keyWords),na=False)]
#FY20_Codes

In [12]:
#Merge the findings and only want the solications that showed up for all three of the search functions
relevantSol_final= relevantSol_keyWords.merge(relevantSol_Codes, how='inner')
relevantSol_final

,NoticeId,Title,Sol#,Department/Ind.Agency,CGAC,Sub-Tier,FPDS Code,Office,AAC Code,PostedDate,...,OrganizationType,State,City,ZipCode,CountryCode,AdditionalInfoLink,Link,Description,year,month
0,935d153a1f54463f84757df2e621b48c,"National Guard Bureau, Region 5, Architect and...",W9133L-21-R-6005,DEPT OF DEFENSE,21.0,DEPT OF THE ARMY,2100,W39L USA NG READINESS CENTER,W9133L,2021-06-30 22:44:52.743-04,...,OFFICE,VA,ARLINGTON,22204-1382,USA,NaN,https://beta.sam.gov/opp/935d153a1f54463f84757...,30 June 2021:† †See attached Amendment W9133L-...,2021,6
1,6c28f17e3eb1443e88d4efaa58d9eaed,Fort Bragg DPW Vertical Construction,W91247-20-R-0009,DEPT OF DEFENSE,21.0,DEPT OF THE ARMY,2100,W6QM MICC FDO FT BRAGG,W91247,2021-06-30 19:38:25.374-04,...,OFFICE,NC,FORT BRAGG,28310-5000,USA,NaN,https://beta.sam.gov/opp/6c28f17e3eb1443e88d4e...,"DPW ñ FBís mission is to sustain, restore, and...",2021,6
2,3153d04985564b0ba2a174751759f457,FULL SERVICE MAINTENANCE TO THE ENSIGHT VELOCI...,N0025921Q0053,DEPT OF DEFENSE,17.0,DEPT OF THE NAVY,1700,NAVAL MEDICAL CENTER SAN DIEGO CA,N00259,2021-06-30 18:35:32.214-04,...,OFFICE,CA,SAN DIEGO,92134-5000,USA,NaN,https://beta.sam.gov/opp/3153d04985564b0ba2a17...,NOTICE TO CONTRACTOR This is a combined synops...,2021,6
3,e07106220572403b9c4016931f820f49,Z2DA--BUILDING 141 INTERIOR STAIR RENOVATIONS ...,36C25921Q0568,"VETERANS AFFAIRS, DEPARTMENT OF",36.0,"VETERANS AFFAIRS, DEPARTMENT OF",3600,NETWORK CONTRACT OFFICE 19 (36C259),36C259,2021-06-30 18:20:11.155-04,...,OFFICE,CO,Greenwood Village,80111,USA,NaN,https://beta.sam.gov/opp/e07106220572403b9c401...,THIS IS A PRESOLICIATION NOTICE A REQUEST FOR ...,2021,6
4,c1528e585ae341f194513c96e8d1df12,Canon Imaging,36C25821Q0272,"VETERANS AFFAIRS, DEPARTMENT OF",36.0,"VETERANS AFFAIRS, DEPARTMENT OF",3600,258-NETWORK CNTRCT OFF 22G (36C258),36C258,2021-06-30 17:58:08.867-04,...,OFFICE,AZ,MESA,85212,USA,NaN,https://beta.sam.gov/opp/c1528e585ae341f194513...,This request for information (RFI) is issued s...,2021,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,2e915a05f5454b1e91df69b08141f333,USDA Seeks to Lease Office and Related Space i...,57-04025-21-NR,"AGRICULTURE, DEPARTMENT OF",12.0,FARM PRODUCTION AND CONSERVATION BUSINESS CENTER,12D0,FPAC BUS CNTR-MGMT SVS DIV,12FPAR,2021-05-18 09:18:00.697-04,...,OFFICE,DC,WASHINGTON,20250,USA,NaN,https://beta.sam.gov/opp/2e915a05f5454b1e91df6...,The U.S. Department of Agriculture (USDA) seek...,2021,5
1220,e4e3a84259534acd914eb87012ea6775,11th Circuit US Court of Appeals- Tuttle Chamb...,47PE0221R0011,GENERAL SERVICES ADMINISTRATION,47.0,PUBLIC BUILDINGS SERVICE,4740,PBS R4 ACQUISITION MGMT DIVISION,47PE00,2021-05-18 09:09:49.223-04,...,OFFICE,GA,ATLANTA,30303,USA,NaN,https://beta.sam.gov/opp/e4e3a84259534acd914eb...,11TH Circuit US Court of Appeals Chambers Reno...,2021,5
1221,f9904ea7b50a4aaab3454ea945df2e8a,USDA Seeks to Lease Office and Related Space i...,57-56005-21-NR,"AGRICULTURE, DEPARTMENT OF",12.0,FARM PRODUCTION AND CONSERVATION BUSINESS CENTER,12D0,FPAC BUS CNTR-MGMT SVS DIV,12FPAR,2021-05-18 08:55:33.501-04,...,OFFICE,DC,WASHINGTON,20250,USA,NaN,https://beta.sam.gov/opp/f9904ea7b50a4aaab3454...,The U.S. Department of Agriculture (USDA) seek...,2021,5
1222,4abe496410ce4493bf589451210c748c,ENGINEER CONSTRUCTION SERVICES FOR THE REPLACE...,19GE5021R0081,"STATE, DEPARTMENT OF",19.0,"STATE, DEPARTMENT OF",1900,ACQUISITIONS - RPSO FRANKFURT,19GE50,2021-05-18 07:41:36.142-04,...,OFFICE,DC,WASHINGTON,20520,USA,NaN,https://beta.sam.gov/opp/4abe496410ce4493bf589...,PRE-SOLICITATION NOTICE No.: 19GE5021R0081 The...,2021,5


In [13]:
relevantSol_final = pd.merge(relevantSol_final, NAICSCode, how='left', left_on='NaicsCode', right_on = 'NaicsCode') #adding on the product category
relevantSol_final = relevantSol_final.drop_duplicates(subset = 'Link') #drop any duplicates

In [14]:
relevantSol_final = relevantSol_final.sort_values(by=['month'])
relevantSol_final

,NoticeId,Title,Sol#,Department/Ind.Agency,CGAC,Sub-Tier,FPDS Code,Office,AAC Code,PostedDate,...,State,City,ZipCode,CountryCode,AdditionalInfoLink,Link,Description,year,month,PRODCAT
1801,cb37fe3b2e1a44ceb231e402873c958b,ENGINEER CONSTRUCTION SERVICES FOR THE REPLACE...,19GE5021R0081,"STATE, DEPARTMENT OF",19.0,"STATE, DEPARTMENT OF",1900,ACQUISITIONS - RPSO FRANKFURT,19GE50,2021-05-18 07:29:08.755-04,...,DC,WASHINGTON,20520,USA,NaN,https://beta.sam.gov/opp/cb37fe3b2e1a44ceb231e...,PRE-SOLICITATION NOTICE No.: 19GE5021R0081 The...,2021,5,Heating and Cooling
1562,f00e746c47a8408dba3e6d7fe212a1dd,Solicitation Notice - LBJ Consolidation & Swit...,47PM0121R0013,GENERAL SERVICES ADMINISTRATION,47.0,PUBLIC BUILDINGS SERVICE,4740,PBS R11 CONSTR CTR 1,47PM01,2021-05-26 09:40:05.186-04,...,DC,WASHINGTON,20407,USA,NaN,https://beta.sam.gov/opp/f00e746c47a8408dba3e6...,AMENDMENT 47PM0121R0013_0002†ADDS THE FOLLOWIN...,2021,5,Heating and Cooling
1560,1a4a71847be240d9a817a17c0bd24a7f,Tactical Equipment Maintenance Facility (TEMF)...,W9126G21R0068,DEPT OF DEFENSE,21.0,DEPT OF THE ARMY,2100,US ARMY ENGINEER DISTRICT FT WORTH,W9126G,2021-05-26 09:58:23.113-04,...,TX,FORT WORTH,76102-6124,USA,NaN,https://beta.sam.gov/opp/1a4a71847be240d9a817a...,This solicitation to be issued on / around 27 ...,2021,5,Heating and Cooling
1558,55819a3f87904b97ad5e9d56bd69b4e0,"Z1DA--442-21-009 Construction, Replace Inacces...",36C25921Q0364,"VETERANS AFFAIRS, DEPARTMENT OF",36.0,"VETERANS AFFAIRS, DEPARTMENT OF",3600,NETWORK CONTRACT OFFICE 19 (36C259),36C259,2021-05-26 10:19:02.411-04,...,CO,Greenwood Village,80111,USA,NaN,https://beta.sam.gov/opp/55819a3f87904b97ad5e9...,THIS IS A PRESOLICIATION NOTICE A REQUEST FOR ...,2021,5,Heating and Cooling
1556,a48a6ab969044420a1ec88b897ba67d6,Gignoux USMS Paint & Carpet Project,47PB0221R0006,GENERAL SERVICES ADMINISTRATION,47.0,PUBLIC BUILDINGS SERVICE,4740,R1 NORTH SERVICE CENTER-VT,47PB04,2021-05-26 10:49:40.737-04,...,VT,BURLINGTON,05401,USA,NaN,https://beta.sam.gov/opp/a48a6ab969044420a1ec8...,This solicitation only authorizes the transmit...,2021,5,Heating and Cooling
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,c08e9a7396ce4aee88be71939ced4496,Z1DA-- Project 621-331- Expand Emergency Depar...,36C24921B0008,"VETERANS AFFAIRS, DEPARTMENT OF",36.0,"VETERANS AFFAIRS, DEPARTMENT OF",3600,249-NETWORK CONTRACT OFFICE 9 (36C249),36C249,2021-06-21 12:25:16.763-04,...,TN,MURFREESBORO,37129,USA,NaN,https://beta.sam.gov/opp/c08e9a7396ce4aee88be7...,The James H. Quillen VA Medical Center (JHQVAM...,2021,6,Heating and Cooling
621,c4296629a99a48b5ab7b99b4e2aff686,"Renovate Squadron Operations Area, JSF Hangar ...",FA282321R0016-Solicitation,DEPT OF DEFENSE,57.0,DEPT OF THE AIR FORCE,5700,FA2823 AFTC PZIO,FA2823,2021-06-21 12:51:53.989-04,...,FL,EGLIN AFB,32542-5418,USA,NaN,https://beta.sam.gov/opp/c4296629a99a48b5ab7b9...,"Eglin Air Force Base (AFB), FL has a requireme...",2021,6,Heating and Cooling
619,3edbeae2a3214972ade84068133f9fe3,"Renovate Squadron Operations Area, JSF Hangar ...",FA282321R0016-Solicitation,DEPT OF DEFENSE,57.0,DEPT OF THE AIR FORCE,5700,FA2823 AFTC PZIO,FA2823,2021-06-21 12:56:05.036-04,...,FL,EGLIN AFB,32542-5418,USA,NaN,https://beta.sam.gov/opp/3edbeae2a3214972ade84...,The proposal due date for this solicitation is...,2021,6,Heating and Cooling
631,feffc40e72fb484299b873650948dc48,J065--Dialysis PM Service,36C24721Q0876,"VETERANS AFFAIRS, DEPARTMENT OF",36.0,"VETERANS AFFAIRS, DEPARTMENT OF",3600,247-NETWORK CONTRACT OFFICE 7 (36C247),36C247,2021-06-21 11:47:21.356-04,...,GA,AUGUSTA,30904,USA,NaN,https://beta.sam.gov/opp/feffc40e72fb484299b87...,Page 1 of 1 NOTICE OF INTENT TO SOLE SOURCE RE...,2021,6,IT & Electronics


In [15]:
#relevantSol_final['Sol#'].to_csv(r'SAM_export/output/Sol_number1.csv', index = None, header = True) #savings the solicitation numbers into an excel, which 
#we will use next time to filter out the already processed solicitations

In [16]:
relevantSol_final = relevantSol_final.loc[~relevantSol_final['Sol#'].isin(existingSol['Sol#'])] #DO NOT run this for the first time in a FY
#finding the soliciations that we found in the last run and will not include them in the new filtered excel 
relevantSol_final

,NoticeId,Title,Sol#,Department/Ind.Agency,CGAC,Sub-Tier,FPDS Code,Office,AAC Code,PostedDate,...,State,City,ZipCode,CountryCode,AdditionalInfoLink,Link,Description,year,month,PRODCAT
1801,cb37fe3b2e1a44ceb231e402873c958b,ENGINEER CONSTRUCTION SERVICES FOR THE REPLACE...,19GE5021R0081,"STATE, DEPARTMENT OF",19.0,"STATE, DEPARTMENT OF",1900,ACQUISITIONS - RPSO FRANKFURT,19GE50,2021-05-18 07:29:08.755-04,...,DC,WASHINGTON,20520,USA,NaN,https://beta.sam.gov/opp/cb37fe3b2e1a44ceb231e...,PRE-SOLICITATION NOTICE No.: 19GE5021R0081 The...,2021,5,Heating and Cooling
1556,a48a6ab969044420a1ec88b897ba67d6,Gignoux USMS Paint & Carpet Project,47PB0221R0006,GENERAL SERVICES ADMINISTRATION,47.0,PUBLIC BUILDINGS SERVICE,4740,R1 NORTH SERVICE CENTER-VT,47PB04,2021-05-26 10:49:40.737-04,...,VT,BURLINGTON,05401,USA,NaN,https://beta.sam.gov/opp/a48a6ab969044420a1ec8...,This solicitation only authorizes the transmit...,2021,5,Heating and Cooling
1555,3a519ac7c82744e1b73805c9d265810d,Request for Information (RFI) - AN/USQ-82(V) x...,N00024-22-R-4103,DEPT OF DEFENSE,17.0,DEPT OF THE NAVY,1700,NAVSEA HQ,NaN,2021-05-26 11:31:24.832-04,...,NaN,NaN,NaN,NaN,NaN,https://beta.sam.gov/opp/3a519ac7c82744e1b7380...,DISCLAIMERS This Request for Information (RFI)...,2021,5,IT & Electronics
1552,da70e8409ff2481caeb5d0d65442e76e,"Renovate Squadron Operations Area, JSF Hangar ...",FA282321R0016-Solicitation,DEPT OF DEFENSE,57.0,DEPT OF THE AIR FORCE,5700,FA2823 AFTC PZIO,FA2823,2021-05-26 13:07:56.416-04,...,FL,EGLIN AFB,32542-5418,USA,NaN,https://beta.sam.gov/opp/da70e8409ff2481caeb5d...,"Eglin Air Force Base (AFB), FL has a requireme...",2021,5,Heating and Cooling
1550,8765b3fd105545ea90d767a20e429ffa,6640--Genomics PA02,36C77621Q0260,"VETERANS AFFAIRS, DEPARTMENT OF",36.0,"VETERANS AFFAIRS, DEPARTMENT OF",3600,PCAC HEALTH INFORMATION (36C776),36C776,2021-05-26 13:17:34.789-04,...,OH,INDEPENDENCE,44131,USA,NaN,https://beta.sam.gov/opp/8765b3fd105545ea90d76...,INTRODUCTION The Veterans Health Administratio...,2021,5,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,c08e9a7396ce4aee88be71939ced4496,Z1DA-- Project 621-331- Expand Emergency Depar...,36C24921B0008,"VETERANS AFFAIRS, DEPARTMENT OF",36.0,"VETERANS AFFAIRS, DEPARTMENT OF",3600,249-NETWORK CONTRACT OFFICE 9 (36C249),36C249,2021-06-21 12:25:16.763-04,...,TN,MURFREESBORO,37129,USA,NaN,https://beta.sam.gov/opp/c08e9a7396ce4aee88be7...,The James H. Quillen VA Medical Center (JHQVAM...,2021,6,Heating and Cooling
621,c4296629a99a48b5ab7b99b4e2aff686,"Renovate Squadron Operations Area, JSF Hangar ...",FA282321R0016-Solicitation,DEPT OF DEFENSE,57.0,DEPT OF THE AIR FORCE,5700,FA2823 AFTC PZIO,FA2823,2021-06-21 12:51:53.989-04,...,FL,EGLIN AFB,32542-5418,USA,NaN,https://beta.sam.gov/opp/c4296629a99a48b5ab7b9...,"Eglin Air Force Base (AFB), FL has a requireme...",2021,6,Heating and Cooling
619,3edbeae2a3214972ade84068133f9fe3,"Renovate Squadron Operations Area, JSF Hangar ...",FA282321R0016-Solicitation,DEPT OF DEFENSE,57.0,DEPT OF THE AIR FORCE,5700,FA2823 AFTC PZIO,FA2823,2021-06-21 12:56:05.036-04,...,FL,EGLIN AFB,32542-5418,USA,NaN,https://beta.sam.gov/opp/3edbeae2a3214972ade84...,The proposal due date for this solicitation is...,2021,6,Heating and Cooling
631,feffc40e72fb484299b873650948dc48,J065--Dialysis PM Service,36C24721Q0876,"VETERANS AFFAIRS, DEPARTMENT OF",36.0,"VETERANS AFFAIRS, DEPARTMENT OF",3600,247-NETWORK CONTRACT OFFICE 7 (36C247),36C247,2021-06-21 11:47:21.356-04,...,GA,AUGUSTA,30904,USA,NaN,https://beta.sam.gov/opp/feffc40e72fb484299b87...,Page 1 of 1 NOTICE OF INTENT TO SOLE SOURCE RE...,2021,6,IT & Electronics


In [17]:
relevantSol_final_clean = relevantSol_final.drop_duplicates(subset = ['Sol#'])
relevantSol_final_clean

,NoticeId,Title,Sol#,Department/Ind.Agency,CGAC,Sub-Tier,FPDS Code,Office,AAC Code,PostedDate,...,State,City,ZipCode,CountryCode,AdditionalInfoLink,Link,Description,year,month,PRODCAT
1801,cb37fe3b2e1a44ceb231e402873c958b,ENGINEER CONSTRUCTION SERVICES FOR THE REPLACE...,19GE5021R0081,"STATE, DEPARTMENT OF",19.0,"STATE, DEPARTMENT OF",1900,ACQUISITIONS - RPSO FRANKFURT,19GE50,2021-05-18 07:29:08.755-04,...,DC,WASHINGTON,20520,USA,NaN,https://beta.sam.gov/opp/cb37fe3b2e1a44ceb231e...,PRE-SOLICITATION NOTICE No.: 19GE5021R0081 The...,2021,5,Heating and Cooling
1556,a48a6ab969044420a1ec88b897ba67d6,Gignoux USMS Paint & Carpet Project,47PB0221R0006,GENERAL SERVICES ADMINISTRATION,47.0,PUBLIC BUILDINGS SERVICE,4740,R1 NORTH SERVICE CENTER-VT,47PB04,2021-05-26 10:49:40.737-04,...,VT,BURLINGTON,05401,USA,NaN,https://beta.sam.gov/opp/a48a6ab969044420a1ec8...,This solicitation only authorizes the transmit...,2021,5,Heating and Cooling
1555,3a519ac7c82744e1b73805c9d265810d,Request for Information (RFI) - AN/USQ-82(V) x...,N00024-22-R-4103,DEPT OF DEFENSE,17.0,DEPT OF THE NAVY,1700,NAVSEA HQ,NaN,2021-05-26 11:31:24.832-04,...,NaN,NaN,NaN,NaN,NaN,https://beta.sam.gov/opp/3a519ac7c82744e1b7380...,DISCLAIMERS This Request for Information (RFI)...,2021,5,IT & Electronics
1552,da70e8409ff2481caeb5d0d65442e76e,"Renovate Squadron Operations Area, JSF Hangar ...",FA282321R0016-Solicitation,DEPT OF DEFENSE,57.0,DEPT OF THE AIR FORCE,5700,FA2823 AFTC PZIO,FA2823,2021-05-26 13:07:56.416-04,...,FL,EGLIN AFB,32542-5418,USA,NaN,https://beta.sam.gov/opp/da70e8409ff2481caeb5d...,"Eglin Air Force Base (AFB), FL has a requireme...",2021,5,Heating and Cooling
1550,8765b3fd105545ea90d767a20e429ffa,6640--Genomics PA02,36C77621Q0260,"VETERANS AFFAIRS, DEPARTMENT OF",36.0,"VETERANS AFFAIRS, DEPARTMENT OF",3600,PCAC HEALTH INFORMATION (36C776),36C776,2021-05-26 13:17:34.789-04,...,OH,INDEPENDENCE,44131,USA,NaN,https://beta.sam.gov/opp/8765b3fd105545ea90d76...,INTRODUCTION The Veterans Health Administratio...,2021,5,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,b2eb5f993555479f85f603829d5ef675,Z--REFURBISH 4 USGS RESEARCH LABS,140G0121R0022,"INTERIOR, DEPARTMENT OF THE",14.0,US GEOLOGICAL SURVEY,1434,OFC OF ACQUSITION GRANTS-NATIONAL,140G01,2021-06-21 11:03:16.705-04,...,VA,RESTON,20192,USA,NaN,https://beta.sam.gov/opp/b2eb5f993555479f85f60...,The U.S. Geological Survey (USGS) has a requir...,2021,6,Heating and Cooling
632,328c3bb09add4e0f922d4b2208e147c7,SOLE SOURCE ñ PROFESSIONAL SUPPORT SERVICES FO...,N0016421CB005,DEPT OF DEFENSE,17.0,DEPT OF THE NAVY,1700,NSWC CRANE,N00164,2021-06-21 11:43:55.443-04,...,IN,CRANE,47522-5001,USA,NaN,https://beta.sam.gov/opp/328c3bb09add4e0f922d4...,N00164-21-C-B005 SOLE SOURCE ñ PROFESSIONAL SU...,2021,6,Heating and Cooling
627,69efe1dd1f944e4eb8900dbd4b65deca,United States Department Of Agriculture seeks ...,57-08073-21-FA,"AGRICULTURE, DEPARTMENT OF",12.0,FARM PRODUCTION AND CONSERVATION BUSINESS CENTER,12D0,FPAC BUS CNTR-MGMT SVS DIV,12FPAR,2021-06-21 12:00:14.86-04,...,DC,WASHINGTON,20250,USA,NaN,https://beta.sam.gov/opp/69efe1dd1f944e4eb8900...,The U.S. Department of Agriculture (USDA) seek...,2021,6,Lighting
625,0cd959c5454e4e4993650ef1c8e0fb61,Z1DA--676-18-103 Upgrade Exterior Signage Wayf...,36C25221B0020,"VETERANS AFFAIRS, DEPARTMENT OF",36.0,"VETERANS AFFAIRS, DEPARTMENT OF",3600,252-NETWORK CONTRACT OFFICE 12 (36C252),36C252,2021-06-21 12:20:37.51-04,...,WI,MILWAUKEE,53214,USA,NaN,https://beta.sam.gov/opp/0cd959c5454e4e4993650...,PRE-SOLICITATION NOTICE Solicitation 36C25221B...,2021,6,Heating and Cooling


In [18]:
relevantSol_final_clean.to_excel('SAM_export/output/filtered.xlsx')
#solicitations that need another round of manual review to ensure they are relevant 

In [19]:
#keep adding filtered to the existingSol dataframe 
#DO NOT RUN THESE LINES for the first run of FY
#existingSolNew = pd.read_excel("SAM_export/output/filtered.xlsx") #using the newly founded sol and creating a panda
#existingSolNew = existingSolNew['Sol#'] #extracting only the sol column

existingSolNew = pd.DataFrame(data = relevantSol_final_clean['Sol#'], columns = ['Sol#'])

existingSol = pd.concat([existingSol, existingSolNew])#concatenate the two panda dateframes
existingSol = existingSol.reset_index(drop=True) 
existingSol.to_csv('SAM_export/input/existingSol.csv') #replace the old existingSol file

In [20]:
existingSol

,Sol#
0,19AQMM20Q0173
1,4564DRFL00023
2,FORTBRAGG4THBN3RDSFGDC0001
3,165THINBRIGADE00013
4,N66001-20-R-0024
...,...
2063,140G0121R0022
2064,N0016421CB005
2065,57-08073-21-FA
2066,36C25221B0020
